In [12]:
import json
import pandas as pd

pd.set_option('display.max_columns', None)

with open('source_files/assistance-listings.json') as f:
    assistanceListingsListString = json.load(f)

with open('source_files/dictionary.json') as f:
    dictionaryList = json.load(f)
    
with open('source_files/organizations.json') as f:
    organizationsList = f.read()

# load dictionary values into dataframe
for i in dictionaryList['_embedded']['jSONObjectList']:
    if i['id'] == 'assistance_type':
        allData = []
        for e in i['elements']:
            for s in e['elements']:
                row = {
                    'parent_code': e['code'],
                    'parent_description': e['description'],
                    'parent_element_id': e['element_id'], 
                    'parent_value': e['value'],
                    'code': s['code'],
                    'element_id': s['element_id'],
                    'value': s['value']
                }
                allData.append(row)
        assistanceTypesDf = pd.read_json(json.dumps(allData)).set_index('element_id')
    if i['id'] == 'applicant_types':
        applicantTypesDf = pd.read_json(json.dumps(i['elements'])).set_index('element_id')
    if i['id'] == 'assistance_usage_types':
        assistanceUsageTypesDf = pd.read_json(json.dumps(i['elements'])).set_index('element_id')
    if i['id'] == 'beneficiary_types':
        beneficiaryTypesDf = pd.read_json(json.dumps(i['elements'])).set_index('element_id')


# load organization values into dataframe
organizationsDf = pd.read_json(organizationsList).set_index('orgKey')

# load assistance listing values
assistanceListingsDf = pd.json_normalize(assistanceListingsListString)

# designate the columns we are interested in and will maintain in the data model
allowedColumnsList = {
    'parentProgramId',
    'latest',
    'revision',
    'fiscalYearLatest',
    'id',
    'data.title',
    'data.website',
    'data.financial.accounts',
    'data.financial.obligations',
    'data.financial.isFundedCurrentFY',
    'data.objective',
    'data.fiscalYear',
    'data.description',
    'data.eligibility.applicant.types',
    'data.eligibility.beneficiary.types',
    'data.eligibility.beneficiary.isSameAsApplicant',
    'data.eligibility.assistanceUsage.types',
    'data.programNumber',
    'data.organizationId',
    'data.alternativeNames',
    'data.assistanceTypes',
    'data.relatedPrograms'
}

# remove unnecessary columns
assistanceListingsDf = assistanceListingsDf.drop(columns=set(assistanceListingsDf)-allowedColumnsList)

In [ ]:
# this section could be cleaned up; originally used for setting up / loading into memory "one dataframe to rule them all"
# for data anlaysis, but some of this computation is unnecessary for the current use case of exporting data to CSV

# transform and explode rows, to build out a comprehensive and flattened data model
assistanceListingsDf = assistanceListingsDf.explode('data.eligibility.applicant.types', ignore_index=True) \
                    .explode('data.assistanceTypes', ignore_index=True) \
                    .explode('data.eligibility.beneficiary.types', ignore_index=True) \
                    .explode('data.eligibility.assistanceUsage.types', ignore_index=True) \
                    .explode('data.financial.obligations', ignore_index=True) \
                    .explode('data.financial.accounts', ignore_index=True) \
                    .explode('data.relatedPrograms', ignore_index=True)
assistanceListingsDf['data.financial.accounts'] = assistanceListingsDf['data.financial.accounts'].apply(lambda x: x.get('code') if x is not None else None)
assistanceListingsDf['data.financial.obligations.isRecoveryAct'] = assistanceListingsDf['data.financial.obligations'].apply(lambda x: x.get('isRecoveryAct') if x is not None else None)
assistanceListingsDf['data.financial.obligations.obligationId'] = assistanceListingsDf['data.financial.obligations'].apply(lambda x: x.get('obligationId') if x is not None else None)
assistanceListingsDf['data.financial.obligations.assistanceType'] = assistanceListingsDf['data.financial.obligations'].apply(lambda x: x.get('assistanceType') if x is not None else None)
assistanceListingsDf['data.financial.obligations.description'] = assistanceListingsDf['data.financial.obligations'].apply(lambda x: x.get('description') if x is not None else None)
assistanceListingsDf['data.financial.obligations.values'] = assistanceListingsDf['data.financial.obligations'].apply(lambda x: x.get('values') if x is not None else None)
assistanceListingsDf = assistanceListingsDf.explode('data.financial.obligations.values', ignore_index=True)
assistanceListingsDf['data.financial.obligations.values.year'] = assistanceListingsDf['data.financial.obligations.values'].apply(lambda x: x.get('year') if x is not None else None)
assistanceListingsDf['data.financial.obligations.values.estimate'] = assistanceListingsDf['data.financial.obligations.values'].apply(lambda x: x.get('estimate') if x is not None else 0)
assistanceListingsDf['data.financial.obligations.values.actual'] = assistanceListingsDf['data.financial.obligations.values'].apply(lambda x: x.get('actual') if x is not None else 0)

# convert datatypes
assistanceListingsDf['data.eligibility.applicant.types'] = pd.to_numeric(assistanceListingsDf['data.eligibility.applicant.types'])
assistanceListingsDf['data.financial.obligations.assistanceType'] = pd.to_numeric(assistanceListingsDf['data.financial.obligations.assistanceType'])
assistanceListingsDf['data.eligibility.assistanceUsage.types'] = pd.to_numeric(assistanceListingsDf['data.eligibility.assistanceUsage.types'])
assistanceListingsDf['data.eligibility.beneficiary.types'] = pd.to_numeric(assistanceListingsDf['data.eligibility.beneficiary.types'])
assistanceListingsDf['data.organizationId'] = pd.to_numeric(assistanceListingsDf['data.organizationId'])

# join in different datasets
assistanceListingsDf = assistanceListingsDf.join(applicantTypesDf, on='data.eligibility.applicant.types', rsuffix='_applicant')
assistanceListingsDf = assistanceListingsDf.join(assistanceTypesDf, on='data.financial.obligations.assistanceType', rsuffix='_assistanceTypes')
assistanceListingsDf = assistanceListingsDf.join(assistanceUsageTypesDf, on='data.eligibility.assistanceUsage.types', rsuffix='_assistanceUsage')
assistanceListingsDf = assistanceListingsDf.join(beneficiaryTypesDf, on='data.eligibility.beneficiary.types', rsuffix='_beneficiary')
assistanceListingsDf = assistanceListingsDf.join(organizationsDf, on='data.organizationId', rsuffix='_organization')

In [5]:
""" programs """
# designate the columns we are interested in and will maintain in the data model
allowedColumnsList = {
    'data.programNumber',
    'data.title',
    'data.description',
    'data.objective',
    'data.organizationId',
    'l1ShortName',
    'l1Name',
    'l2Name'
}
# remove unnecessary columns and save to csv
assistanceListingsDf.drop(columns=set(assistanceListingsDf)-allowedColumnsList).drop_duplicates(ignore_index=True).to_csv('data_extracts/db_programs.csv', index=False)

In [ ]:
""" program_to_applicant_type """
# designate the columns we are interested in and will maintain in the data model
allowedColumnsList = {
    'data.programNumber',
    'value'
}
# remove unnecessary columns and save to csv
assistanceListingsDf.drop(columns=set(assistanceListingsDf)-allowedColumnsList).drop_duplicates(ignore_index=True).to_csv('data_extracts/db_program_to_applicant_type.csv', index=False)

In [ ]:
""" program_to_category """
# designate the columns we are interested in and will maintain in the data model
allowedColumnsList = {
    'data.programNumber',
    'value_assistanceUsage'
}
# remove unnecessary columns and save to csv
assistanceListingsDf.drop(columns=set(assistanceListingsDf)-allowedColumnsList).drop_duplicates(ignore_index=True).to_csv('data_extracts/db_program_to_category.csv', index=False)

In [ ]:
""" program_to_beneficiary """
# designate the columns we are interested in and will maintain in the data model
allowedColumnsList = {
    'data.programNumber',
    'value_beneficiary'
}
# remove unnecessary columns and save to csv
assistanceListingsDf.drop(columns=set(assistanceListingsDf)-allowedColumnsList).drop_duplicates(ignore_index=True).to_csv('data_extracts/db_program_to_beneficiary.csv', index=False)

In [ ]:
""" program_to_assistance_type """
# designate the columns we are interested in and will maintain in the data model
allowedColumnsList = {
    'data.programNumber',
    'data.financial.obligations.assistanceType',
    'parent_code',
    'parent_description',
    'parent_element_id',
    'parent_value',
    'code_assistanceTypes',
    'value_assistanceTypes',
    'code_assistanceUsage'
}
# remove unnecessary columns and save to csv
assistanceListingsDf.drop(columns=set(assistanceListingsDf)-allowedColumnsList).drop_duplicates(ignore_index=True).to_csv('data_extracts/db_program_to_assistance_type.csv', index=False)

In [4]:
""" program obligations """
# designate the columns we are interested in and will maintain in the data model
allowedColumnsList = {
    'data.programNumber',
    'data.financial.obligations.obligationId',
    'data.financial.obligations.values.year',
    'data.financial.obligations.values.actual',
    'data.financial.obligations.values.estimate'
}

# remove unnecessary columns and save to csv
assistanceListingsDf.drop(columns=set(assistanceListingsDf)-allowedColumnsList).drop_duplicates(ignore_index=True).groupby(['data.programNumber','data.financial.obligations.values.year']).agg({'data.financial.obligations.values.estimate':['sum'], 
                         'data.financial.obligations.values.actual':'sum'}).reset_index().to_csv('data_extracts/db_program_obligations.csv', index=False)